In [2]:
# sources : 
# https://medium.com/swlh/lets-write-a-chat-app-in-python-f6783a9ac170
# Chatbox :
# http://code.activestate.com/recipes/580757-chatbox-megawidget-for-tkinter/

from tkinter import font, messagebox
import tkinter
import tkinter.ttk as tk
from Chatbox import Chatbox
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(hosts=["https://70fa05be281c4ff8977b2a9e557f7690.westeurope.azure.elastic-cloud.com:9243/"], http_auth=("elastic","dpucymmIkDh7iOMjbUFFaRqA"))

a=helpers.scan(es,query={"query":{"match_all": {}}},scroll='1m',index='zoo_v4')#like others so far

names=[aa['_source']['name'] for aa in a]

def exit():
    root.destroy()

def show_game_frame():
    home_menu.pack_forget()
    selection_frame.pack_forget()
    lb_frame.pack_forget()
    game_frame.pack(fill=tkinter.BOTH, expand=1)
    
def show_home_frame():
    game_frame.pack_forget()
    selection_frame.pack_forget()
    lb_frame.pack_forget()
    home_menu.pack(fill=tkinter.BOTH, expand=1)
    
def show_selection_frame():
    game_frame.pack_forget()
    home_menu.pack_forget()
    lb_frame.pack(padx=(80,80))
    selection_frame.pack(fill=tkinter.BOTH, expand=1)

def doubleclick(event): 
    cs = listbox.curselection()
    selected_animal.set(listbox.get(cs))
    show_game_frame()

def selected_cb(var, indx, mode):
    display_animal.set('Describe selected animal: '+selected_animal.get())
    
################################
#   Main program starts here   #
################################   
    
root = tkinter.Tk()
root.title("Animal Guesser")
root.geometry('450x550')

# Define styling
s = tk.Style()
s.configure("exit.TButton", font=('Arial','18','bold'))
s.configure("start.TButton",   font=('Arial','18','bold'))
s.map('exit.TButton',foreground=[('pressed', 'dark red'),('active', 'red')], background=[('pressed', 'red'),('active', 'red')])
s.map('start.TButton',foreground=[('pressed', 'dark green'),('active', 'green')], background=[('pressed', 'green'),('active','green')])

s.configure("G.TLabel",     font=('Arial','20','bold'), foreground='green')
s.configure("G.Sub.TLabel", font=('Arial','15','bold'), foreground='dark green')
s.configure("G.Desc.TLabel", font=('Helvetica','13'))
s.configure('TFrame', background='white')
s.configure('TLabel', background='white')

home_menu=tk.Frame(root)
home_menu.pack(fill=tkinter.BOTH, expand=1)
game_frame=tk.Frame(root)
selection_frame=tk.Frame(root)

# building home menu (FRAME)
tk.Label(home_menu,text = 'Animal Guesser', style='G.TLabel').pack(pady=(30,0))
tk.Label(home_menu, text = 'A game based on the Taboo board game..', style='G.Sub.TLabel').pack(pady=(5,50))
tk.Label(home_menu, text = 'The objective of this game is to describe an animal.', style='G.Desc.TLabel').pack(pady=(20,5))
tk.Label(home_menu, text = 'After every single description the AI will make a guess.', style='G.Desc.TLabel').pack(pady=(0,5))
tk.Label(home_menu, text = 'If the guess is correct you can answer with "Yes",', style='G.Desc.TLabel').pack(pady=(0,5))
tk.Label(home_menu, text = 'if the guess is wrong you must answer with "No"', style='G.Desc.TLabel').pack(pady=(0,5))
tk.Label(home_menu, text = 'and provide a following description.', style='G.Desc.TLabel').pack()

tk.Button(home_menu,text='QUIT', width=18, style='exit.TButton', command = exit).pack(pady=(20,60),side=tkinter.BOTTOM)
tk.Button(home_menu,text='PLAY', width=18, style='start.TButton', command = show_selection_frame).pack(pady=(0,20),side=tkinter.BOTTOM)

# building animal selection (FRAME)
selected_animal = tkinter.StringVar()
display_animal = tkinter.StringVar()
selected_animal.trace_add("write", selected_cb)
tk.Label(selection_frame,text = 'Pick an animal to describe', style='G.TLabel', font=('Arial','14','bold')).pack(pady=(30,5))
tk.Label(selection_frame,text = 'Double click to select', style='G.TLabel', font=('Arial','14')).pack(pady=(5,15))

lb_frame=tk.Frame(selection_frame)
listbox = tkinter.Listbox(lb_frame,height=16,width=22,bg="white",activestyle='dotbox',font="Helvetica 16",fg="black")
listbox.insert("end", *names)
listbox.configure(justify=tkinter.CENTER)
listbox.bind('<Double-1>', doubleclick) 

scrollbar = tk.Scrollbar(lb_frame) 
scrollbar.pack(side = tkinter.RIGHT, fill = tkinter.BOTH) 
scrollbar.config(command = listbox.yview)

listbox.config(yscrollcommand = scrollbar.set) 
listbox.pack(side=tkinter.RIGHT,fill=tkinter.Y)

# building game (FRAME)
tk.Label(game_frame,textvariable=display_animal, style='Game.TLabel').pack(pady=(10,10))

# create tag to pass to Chatbox to style AI/player text
tags = {
    'player':{'font':'Helvetica 10'},
    'computer':{'font':'Helvetica 10 bold italic'}
}
chatbox = Chatbox(game_frame, tags=tags, history_padx=20, history_pady=15)
chatbox.user_message("AI", "Hi there player.","computer")
chatbox.user_message("PLAYER","Hi there computer!","player")
chatbox.interior.pack(expand=True, fill=tkinter.BOTH)

b_home = tk.Button(game_frame,text='End Game', width=15, style='exit.TButton', command = show_home_frame).pack(pady=(10,10))

root.mainloop()